### Perceptual Decision Making
Sudeshna Bora & Leonard Hollander

$\textbf{Exercise 13.1: the network implementation}$


Check

$\textbf{Exercise 13.1.1. Question: Understanding Brian2 Monitors}$

Variable names are $A, B, Z$  and $\textbf{inhib}$


The names of the different monitors are simply the denoted by adding the name of the population to the prefixes: $\textbf{"spike_monitor_ "}$, $\textbf{"rate_monitor_ "}$ and $\textbf{"voltage_monitor_ "}$, where the name of the population is specified in the end.
    

The state monitor records the temporal evolution of the neuron voltage $\textbf{v}$

In [ ]:
import brian2 as b2
from neurodynex.tools import plot_tools
from neurodynex.competing_populations import decision_making as DM
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

$\textbf{13.1.2. Question: Accessing a dictionary to plot the population rates}$


In [ ]:
#define experiments parameters:
trial_time = 800 * b2.ms
#running simulation:
results = DM.sim_decision_making_network(t_stimulus_start= 50. * b2.ms,\
                                                      coherence_level=-0.6, max_sim_time=1000. * b2.ms)                                                    

In [ ]:
avg_wds = np.zeros(11)
avg_wds[:-1] = np.arange(.1,5.1,.5) #different window sizes
avg_wds[-1] = 50
#plotting results:
def plot_sim_results(avg_wdw: float, results:dict=results) -> None:
    
    print('Simulation with window width: {}ms'.format(avg_wdw))
    plot_tools.plot_network_activity(results["rate_monitor_A"], results["spike_monitor_A"],
                                 results["voltage_monitor_A"], t_min=0. * b2.ms, avg_window_width=avg_wdw *b2.ms,
                                 sup_title="Left with width {}ms".format(avg_wdw))
    
    plot_tools.plot_network_activity(results["rate_monitor_B"], results["spike_monitor_B"],
                                 results["voltage_monitor_B"], t_min=0. * b2.ms, avg_window_width=avg_wdw *b2.ms,
                                 sup_title="Right with width {}ms".format(avg_wdw))
    
    plot_tools.plot_network_activity(results["rate_monitor_inhib"], results["spike_monitor_inhib"],
                                 results["voltage_monitor_inhib"], t_min=0. * b2.ms, avg_window_width=avg_wdw *b2.ms,
                                 sup_title="Inhib with width {}ms".format(avg_wdw))
    
    plot_tools.plot_network_activity(results["rate_monitor_Z"], results["spike_monitor_Z"],
                                 results["voltage_monitor_Z"], t_min=0. * b2.ms, avg_window_width=avg_wdw *b2.ms,
                                 sup_title="Z with width {}ms".format(avg_wdw))
    

    plt.show()
    
    return None


for idx, avg_wdw in np.ndenumerate(avg_wds):
    
    plot_sim_results(avg_wdw)

$\textbf{Answer:}$ We can see that as we increase the average window size, the firing rates become more continuous looking curves. It's fairly intuitive that at small window sizes the firing rates approximate $\delta$ pulses, since, on average, only single spikes fall into a single window. At the other extreme, taking a window size equal to the trial lenght would be equivalent to recording the mean spike count rate, i.e. the number of spikes occuring over the time of the trial, divided by that time.


At $5ms$ the overall curves already look pretty reasonable in our eyes. Qualitatively, they match our expectations, which is that after a while the firing rates in the specific neural populations $A$ and $B$ tend to either a continuous firing activity or silence. At the prescribed coherence level the right population should show heightened activity while the left population should 'quiet down'. 

Taking the average across much larger intervals smoothens out the curves considerable to reveal long-term trends. At $50ms$ we can clearly see the ramp-up and attenuation play out on a relatively large timescale. Both kinds of information are valuable, however. In partciular, the choice of an adequate decision criterion in later parts of the exercise will depend on the choice of window size. We think it best to use intermediate windowsizes, because they balance locally available information (e.g. information about short spike "bursts") against long term trends.

$\textbf{13.2. Exercise: Stimulating the decision makin circuit}$

$\textbf{13.2.1. Question: Coherence Level}$

$\begin{equation}
\mu_{left} - \mu_{right} = \mu_0 \cdot c
\end{equation}$

Since the difference of two Gaussian is a Gaussian with 'subtracted' means and 'added' variances we have

$\begin{equation}
\mu_{left} - \mu_{right} \sim \mathcal{N}(\mu_{left} - \mu_{right}, 2\sigma2)
\end{equation}$

$\textbf{Default values:}$ The default values are 160 and 20 Hz respectively for $\mu_0$ and $\sigma$. With these values we can calculate $\mu_{left}$ and $\mu_{right}$ analytically from the equations above: 


\begin{align*}
\mu_{left} &= 64Hz \\
\mu_{right} &= 96Hz
\end{align*}


At $c=-0.2$, we have $\mu_{left} - \mu_{right}$=-32Hz (indicating we will have a decision favouring the right population). The variance $\sigma=40Hz$. We can thus see that the variance is quite large compared with the mean, indicating that noise can perturb the system towards the wrong decision. This is intuitive, given the relatively small coherence $c$ of dot motion. If we had perfect coherence (1/-1) we would have a much larger mean (160Hz). In this case, the system always makes the correct decision.

$\textbf{13.2.2. Question: Input stimuli with different coherence levels}$

Simulations with c=0.6:

In [ ]:
#default window width (from above):
def_window_width = avg_wds[-1] #5ms window

for i in np.arange(5): #run five simulations

    #coherence level -.1:
    results01=DM.sim_decision_making_network(t_stimulus_start= 50. * b2.ms,\
                                                      coherence_level=.6, max_sim_time=1000. * b2.ms)   
    plot_sim_results(def_window_width, results01)

Simulations with c=-0.1:

In [ ]:
for i in np.arange(5): #run five simulations

    #coherence level -.1:
    results06=DM.sim_decision_making_network(t_stimulus_start= 50. * b2.ms,\
                                                      coherence_level=-0.1, max_sim_time=1000. * b2.ms)

    plot_sim_results(def_window_width, results06)

We already mentioned that at low coherence levels, the system should be expected, on some occassions, (1) either to not have enough time to settle into one of the attractors or (2) possibly even make the wrong decision. In some trials, most notably the first and the last), while we do see the correct ramp-up/attenuation of firing rates in the two populations respectively, it seems as though the activity in the right subpopulation might not (depending on the stringency of our decision criterion) be large enough to say that the system made a decision. 

In [ ]:
avg_window_width = 123*b2.ms
sr = results["rate_monitor_A"].smooth_rate(window="flat", width=avg_window_width)/b2.Hz

In [ ]:
def plot_dSpace (r_mon1, 
                 r_mon2,
                 xlabel: str= 'Firing rate left',
                 ylabel: str= 'Firing rate right',
                 avg_window_width=avg_window_width) -> None:
    
     #Get time points to colour code temporal progression of firing rate

     m1 = r_mon1.smooth_rate(window="flat", width=avg_window_width)
     m2 = r_mon2.smooth_rate(window="flat", width=avg_window_width)

    
     #Get time points to colour code temporal progression of firing rate
     time_ax = np.linspace(0,trial_time / b2.ms, len(m1))
     #Plot the results:
     plt.figure()
     plt.scatter(m1,m2,c=time_ax, cmap='coolwarm')
     plt.title('Decision Space for width={}s'.format(avg_window_width))
     plt.xlabel(xlabel)
     plt.ylabel(ylabel)
     plt.colorbar(label='Time in $ms$')
     
     #plt.show()

     return None

#Plot decision space for various window_widths

avg_wds = (1,5,20,50,100)
for wdw in avg_wds:
    
    plot_dSpace(results["rate_monitor_A"],\
        results["rate_monitor_B"], avg_window_width=wdw * b2.ms)
    


We think this nicely illustrates something we already mentioned earlier, namely that an increased window size allows the firing rates to look more like a smooth function rather than a series of $\delta$ spikes. 

In this case we can clearly see that the system settles in the right attractor, i.e. the upper left corner of the decision space. Initially, the firing rate on the left side increases, then the firing rate of the right population begins to increase, inhibiting the left population driving the system to the upper left corner. The rate at which this corner is approached seems to increase with time. Presumably this is because of exisiting positive feedback loops. The right population inhibits the the left population blocking its own inhibition by said left population, thereby increasing its own activity.

$\textbf{13.3.2. Question: Implementing a decision criterion}$

Initially, we may try with a constant threshhold that we derive empirically from our experiemnts above. In those it always seemed that the 'favoured' subpopulation settled in a frequency band around 50Hz (excited) and the other subpopulation's activity was attenuated successively to eventually become 0. If we apply this decision criterion, then our threshhold should be something like 50Hz. But this will only work for window sizes that are in the ballpark of what we used earlier. If we choose the windowsize too large, our result have a lot of global, but very little local information.

One could also try to determine the threshold analytically by taking performing a one-sided t-Test on intervals of length $\Delta t$ (window size; p-value can freely adjustable) and seeing whether that test ever allowed one to discard the null-hypthosis (firing rate  $\sim \mu_0$). 

In [ ]:
from typing import Optional, Tuple

def get_decision_time (r_mon1,
                      r_mon2,
                      avg_window_width=5*b2.ms,
                      rthresh=50*b2.Hz):
    
    
    """
    Computes the decision time:
    @params:
    r_mon1: rate monitor
    r_mon2: rate monitor
    avg_window_width: window_width across 
    whicht spikes are averaged
    rthresh: firing rate threshold (decision?)
    
    """
    
    m1 = r_mon1.smooth_rate(window="flat", width=avg_window_width)
    m2 = r_mon2.smooth_rate(window="flat", width=avg_window_width)
    
    #Filter by threshold:
    m1, m2 = np.where(m1 > rthresh)[0], np.where(m2 > rthresh)[0]
    
    #make sure decision is made
    assert(len(m1)==0 or len(m2)==0), "Decision Failure: check rate threshhold"
    
    return (0 if len(m1)==0 else np.min(m1) * b2.defaultclock.dt,\
            0 if len(m2)==0 else np.min(m2) * b2.defaultclock.dt)

get_decision_time(results["rate_monitor_B"],\
                  results["rate_monitor_A"])

       

$\textbf{13.4.1. Question: Running multiple simulations}$

In [ ]:
import pandas as pd

def perc_corr(coherence_levels,
             time_to_A,
             time_to_B,
             count_A,
             count_B,
             count_No,
             no_decisions: bool=True):
    
    """
    Computes the percentage of correct and incorrect decisions:
    @params:
    coherence_levels: a list of coherence levels
    time_to_A: time stamps of decisions for A
    time_to_B: time stamps of decisions for B
    count_A: no of times decision A was made
    count_B: no of times decision B was made
    no_decisions: determines whether refraining should
    always count as having made an incorrect decision.
    
    
    """
    
    nLevels, nReps = time_to_A.shape
    result = np.zeros((nLevels, 2))
    
    
    for idx, coLev in np.ndenumerate(coherence_levels):
        
        i = 0 if coLev > 0 else 1 #A is correct when coLev > 1, otherwise B is correct
        
        #count correct and incorrect decisions
        times_correct, times_wrong = (count_A[idx], count_B[idx])[::np.power(-1,i)]
        
        #Calculate percentages
        if no_decisions:
            result[idx,:] = np.array([times_correct, nReps - times_correct]) / nReps
        
        else:
            count_reps = (nReps - count_No)[idx] #count valid trials
            result[idx,:] = np.array([times_correct, times_wrong]) / count_reps
        
    return result

#Percentages when withholding a decision isn't counted:
percentages = perc_corr(coherence_levels, 
             time_to_A, 
             time_to_B, 
             count_A, 
             count_B,
             count_No,
             no_decisions=False)

percentages = pd.DataFrame(percentages,\
                          columns=['incorrect', 'correct'][::-1],\
                          index=coherence_levels)

print('Percentages with non-decisions not counted: ')
percentages

In [ ]:
#Percentages when withholding a decision counts as wrong:      
percentages_noDecWrong = perc_corr(coherence_levels, 
             time_to_A, 
             time_to_B, 
             count_A, 
             count_B,
             count_No,
             no_decisions=True)

percentages_noDecWrong = pd.DataFrame(percentages_noDecWrong,\
                          columns=['incorrect', 'correct'][::-1],\
                          index=coherence_levels)

print('Percentages with non-decisions counted as wrong: ')
percentages_noDecWrong

Thus, we see our earlier suspicion confirmed. For small coherence levels, the system actually mostly withholds making a decision. The choice of simulation time, rate threshhold and window size contribute to this result. With a longer integration time, the system may be able to make more (ultimately correct decisions). Choice becomes more unreliable with a smaller rate threshhold. Our choice of threshhold seems to be a rather conserative one, since the system mostly foregoes making a choice. It is surprising therefore that it nonetheless makes quite a lot of wrong decisions. The system here seems to be at chance level. However, we surmise that this is a statistical artefact, due to the limited number of simulations we are able to perform.

At larger levels of coherence the system has a perfect performance when we black out trials in which no decision is made. Overall, the system behaves according to our expectations.

In [ ]:
#define experiment parameters:

coherence_levels = [.15,-.8]
nreps = 15
avg_window_width = 5 * b2.ms
max_sim_time = 1000 * b2.ms
rThresh = 50 * b2.Hz

#run simulation:
time_to_A,time_to_B, count_A, count_B, count_No = \
DM.run_multiple_simulations(get_decision_time,\
                            coherence_levels,\
                            nreps,\
                            max_sim_time=max_sim_time,\
                            rate_threshold = rThresh,\
                            avg_window_width=avg_window_width)



In [ ]:
time_A = time_to_A.sum(axis=1) / count_A
np.where(np.isnan(time_A),0,  time_A)
time_to_B.sum(axis=1) / count_B


In [ ]:
def plot_simulation_stats(coherence_levels, 
                          time_to_A, 
                          time_to_B, 
                          count_A, 
                          count_B) -> None:
    
    """
    Plots a subplot consisting of three graphs:
        1. percentage correct vs. coherence level (no decisions counted)
        2. percentage correct vs. coherence level (no decisions not counted)
        3. Integration time vs. coherence level
        
        @params: ...

    """
    
    #get integration times:
    
    time_A = time_to_A.sum(axis=1) / count_A
    time_A = np.where(np.isnan(time_A), 1,  time_A)
    time_B = time_to_B.sum(axis=1) / count_B
    
    
    fig, axs = plt.subplots(1,3, figsize=(10,5))
    plt.tight_layout(h_pad=20, w_pad=2)

    
    axs[0].set_title('not counting no-decision trials')
    axs[0].plot(coherence_levels, percentages.values[:,0]*100, c='tomato')
    axs[0].set_xlabel('coherence level $c$')
    axs[0].set_ylabel('correct decisions in $\%$')
    axs[1].set_title('counting no-decision trials')
    axs[1].plot(coherence_levels, percentages_noDecWrong.values[:,0]*100, c='tomato')
    axs[1].set_xlabel('coherence level $c$')
    axs[1].set_ylabel('correct decisions in $\%$')
    axs[2].set_title('coherence levels vs. integration time ')
    axs[2].plot(coherence_levels, time_A, c='tomato', label='integration time A')
    axs[2].plot(coherence_levels, time_B, c='steelblue', label='integration time B')
    axs[2].set_xlabel('coherence level $c$')
    axs[2].set_ylabel('integration time')
    axs[2].legend()
    
    
    return None

plot_simulation_stats(coherence_levels,
                     time_to_A,
                     time_to_B,
                     count_A,
                     count_B)
  
        
    
    
    
    

These figures summarize the simulation results. Firstly, we can see that the percentage of correct decisions (on any way of counting) decreases as we decrease the coherence. Secondly, we can see that integration time increases. Moreover, looking at the integration time curves we can see that integration time is decreased for incorrect as opposed to correct decisions (since at $c=0.8$ decision A was never made, we capped integration time at 1s for the sake of the figure, but actually it should be understood that the integration time there is $\infty$). This replicates one key finding from the psychophysical experiments on decision making that were already mentioned in the paper.